**PART1: STACKING STABLE MODEL: CatBoost + LogisticRegression**

**10 CatBoost Model + LogisticRegression: Private LB: 0.46014 ----- Public LB: 0.52663**

**14 CatBoost Model + LogisticRegression: Private LB: 0.46035 ---- Public LB: 0.52610**

**Best Single CatBoost Model: Private LB: 0.46084 ---- Public LB: 0.52464 && 0.46087 -- 0.52461**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import lightgbm as lgb
import os
print(os.listdir('../input'))

In [ ]:
train = pd.read_csv('../input/klps-creditscring-challenge-for-students/train.csv')
test = pd.read_csv('../input/klps-creditscring-challenge-for-students/test.csv')

In [ ]:
oof_cat1 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7619.npy')
oof_cat2 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7620.npy') 
oof_cat3 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7620 (1).npy') 
oof_cat4 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7621 (1).npy')  
oof_cat5 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7622 (1).npy')  
oof_cat6 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7625.npy') 
#oof_cat7 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7623.npy')  
oof_cat8 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7623 (1).npy')
oof_cat9 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7624.npy')
oof_cat10 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7624 (1).npy')
oof_cat11 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7625 (1).npy')
oof_cat12 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7625 (2).npy')
#oof_cat13 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7626.npy')
oof_cat14 = np.load('../input/kalapa2-final-blending/oof_cat_full_0.7628.npy')

# part2
oof_cat15 = np.load('../input/kalapa2-final-blending-part2/oof_cat_full_0.7621 (2).npy')
oof_cat16 = np.load('../input/kalapa2-final-blending-part2/oof_cat_full_0.7623 (2).npy')
oof_cat17 = np.load('../input/kalapa2-final-blending-part2/oof_cat_full_0.7624 (2).npy')
oof_cat18 = np.load('../input/kalapa2-final-blending-part2/oof_cat_full_0.7625 (4).npy')
oof_cat19 = np.load('../input/kalapa2-final-blending-part2/oof_cat_full_0.7627 (1).npy')
oof_cat20 = np.load('../input/kalapa2-final-blending-part2/oof_cat_full_0.7629.npy')

pred_cat1 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7619.csv')  
pred_cat2 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7620.csv') 
pred_cat3 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7620 (1).csv') 
pred_cat4 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7621 (1).csv') 
pred_cat5 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7622 (1).csv') 
pred_cat6 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7625.csv') 
#pred_cat7 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7623.csv') 
pred_cat8 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7623 (1).csv')
pred_cat9 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7624.csv')
pred_cat10 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7624 (1).csv')
pred_cat11 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7625 (1).csv')
pred_cat12 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7625 (2).csv')
#pred_cat13 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7626.csv')
pred_cat14 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7628.csv')

pred_cat15 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7621 (2).csv')
pred_cat16 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7623 (2).csv')
pred_cat17 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7624 (2).csv')
pred_cat18 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7625 (4).csv')
pred_cat19 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7627 (1).csv')
pred_cat20 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_7629.csv')

In [ ]:
y = train['label']

f_stack_train = pd.DataFrame({'cat1': oof_cat1.ravel(),
                             'cat2': oof_cat2.ravel(),
                             'cat3': oof_cat3.ravel(),
                             'cat4': oof_cat4.ravel(),
                             'cat5': oof_cat5.ravel(),
                             'cat6': oof_cat6.ravel(),
                             #'cat7': oof_cat7.ravel(),
                              'cat8': oof_cat8.ravel(),
                              'cat9': oof_cat9.ravel(),
                              'cat10': oof_cat10.ravel(),
                              #'cat11': oof_cat11.ravel(),
                              #'cat12': oof_cat12.ravel(),
                              #'cat13': oof_cat13.ravel(),
                              'cat14': oof_cat14.ravel(),
                              #'cat15': oof_cat15.ravel(),
                              #'cat16': oof_cat16.ravel(),
                              #'cat17': oof_cat17.ravel(),
                             # 'cat18': oof_cat18.ravel(),
                              #'cat19': oof_cat19.ravel(),
                              'cat20': oof_cat20.ravel(),
                             })

f_stack_test = pd.DataFrame({'cat1': pred_cat1.label,
                            'cat2': pred_cat2.label,
                            'cat3': pred_cat3.label,
                            'cat4': pred_cat4.label,
                            'cat5': pred_cat5.label,
                            'cat6': pred_cat6.label,
                            #'cat7': pred_cat7.label,
                             'cat8': pred_cat8.label,
                             'cat9': pred_cat9.label,
                             'cat10': pred_cat10.label,
                             #'cat11': pred_cat11.label,
                             #'cat12': pred_cat12.label,
                            #'cat13': pred_cat13.label,
                             'cat14': pred_cat14.label,
                             #'cat15': pred_cat15.label,
                            # 'cat16': pred_cat16.label,
                             #'cat17': pred_cat17.label,
                            # 'cat18': pred_cat18.label,
                             #'cat19': pred_cat19.label,
                             'cat20': pred_cat20.label,
                            })

In [ ]:
f_stack_train

In [ ]:
f_stack_test

In [ ]:
from sklearn.metrics import roc_auc_score

kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
resu1 = 0
y_pred = 0
X_train = f_stack_train
y_train = y

stack_train = np.zeros([X_train.shape[0],])
for train_index, test_index in kf.split(f_stack_train.values, y.values):
    X_train2= X_train.iloc[train_index,:]
    y_train2= y_train.iloc[train_index]
    X_test2= X_train.iloc[test_index,:]
    y_test2= y_train.iloc[test_index]
    clf = LogisticRegression(C=0.1,n_jobs=4,penalty="l2")
    
    clf.fit(X_train2,y_train2)
    
    temp_predict = clf.predict_proba(X_test2)[:, 1]
    stack_train[test_index] = temp_predict
    
    y_pred += clf.predict_proba(f_stack_test)[:, 1] / 5
    
    roc = roc_auc_score(y_test2, temp_predict)
    print(roc)
    resu1 += roc/5
print(f'End:{resu1}')

sub = pd.DataFrame({'id': test.id, 'label': y_pred})
sub.to_csv('stacking_final.csv', index = False)
sub.head(10)

In [ ]:
from sklearn.metrics import roc_auc_score

kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
resu1 = 0
y_pred = 0
X_train = f_stack_train
y_train = y

stack_train = np.zeros([X_train.shape[0],])
for train_index, test_index in kf.split(f_stack_train.values, y.values):
    X_train2= X_train.iloc[train_index,:]
    y_train2= y_train.iloc[train_index]
    X_test2= X_train.iloc[test_index,:]
    y_test2= y_train.iloc[test_index]
    clf = LogisticRegression(C=0.01,n_jobs=4,penalty="l2")
    
    clf.fit(X_train2,y_train2)
    
    temp_predict = clf.predict_proba(X_test2)[:, 1]
    stack_train[test_index] = temp_predict
    
    y_pred += clf.predict_proba(f_stack_test)[:, 1] / 5
    
    roc = roc_auc_score(y_test2, temp_predict)
    print(roc)
    resu1 += roc/5
print(f'End:{resu1}')

sub = pd.DataFrame({'id': test.id, 'label': y_pred})
sub.to_csv('stacking_final2.csv', index = False)
sub.head(10)

In [ ]:
from sklearn.metrics import roc_auc_score

kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
resu1 = 0
y_pred = 0
X_train = f_stack_train
y_train = y

stack_train = np.zeros([X_train.shape[0],])
for train_index, test_index in kf.split(f_stack_train.values, y.values):
    X_train2= X_train.iloc[train_index,:]
    y_train2= y_train.iloc[train_index]
    X_test2= X_train.iloc[test_index,:]
    y_test2= y_train.iloc[test_index]
    #clf = LogisticRegression(C=1,n_jobs=4,penalty="l2")
    clf = ExtraTreesClassifier(n_estimators = 150, random_state=42,
                              max_depth = 3)
    
    clf.fit(X_train2,y_train2)
    
    temp_predict = clf.predict_proba(X_test2)[:, 1]
    stack_train[test_index] = temp_predict
    
    y_pred += clf.predict_proba(f_stack_test)[:, 1] / 5
    
    roc = roc_auc_score(y_test2, temp_predict)
    print(roc)
    resu1 += roc/5
print(f'End:{resu1}')

sub = pd.DataFrame({'id': test.id, 'label': y_pred})
sub.to_csv('stacking_extratree.csv', index = False)
sub.head(10)

**PART2: BLENDING: stable catboost model + non stable catboost model:  (16 + 8) model**

**Private: 0.46028 --- Public: 0.53486**

In [ ]:
#### NON STABLE CATBOOST MODEL
cat1 = pd.read_csv('../input/kalapa2-cat-overfit-new-v1/Cat_full_0.7609.csv') # best LB 0.53792
cat2 = pd.read_csv('../input/kalapa2-cat-overfit-new-v1/Cat_full_0.7607 (4).csv') # 0.53779

cat3 = pd.read_csv('../input/kalapa2-cat-overfit-new-v1/Cat_full_0.7606 (3).csv') # 0.53736
cat4 = pd.read_csv('../input/kalapa2-cat-overfit-new-v1/Cat_full_0.7606 (2).csv') # 0.53711

cat5 = pd.read_csv('../input/kalapa2-cat-overfit-new-v1/Cat_full_0.7608 (6).csv') # 0.5385
cat6 = pd.read_csv('../input/kalapa2-cat-overfit-new-v1/Cat_full_0.7607 (5).csv') # 0.53814

cat7 = pd.read_csv('../input/kalapa2-cat-overfit-new-v1/Cat_full_0.7619 (2).csv') # 0.53415 add more fe

cat8 = pd.read_csv('../input/kalapa2-cat-overfit-new-v1/Cat_full_0.7607 (8).csv') # 0.53745

###### STABLE CATBOOST MODEL

new1 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7625.csv')
new2 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7622 (1).csv')
new3 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7621 (1).csv')
new4 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7620 (1).csv')
new5 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7620.csv')
new6 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7619.csv')

#pred_cat7 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7623.csv') #pd.read_csv('../input/kalapa2-cat-overfit-new-v1/Cat_full_0.7611 (1).csv')
new7 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7623 (1).csv')
new8 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7624.csv')
new9 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7624 (1).csv')
new10 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7625 (1).csv')
new11 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7625 (2).csv')
#pred_cat13 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7626.csv')
new12 = pd.read_csv('../input/kalapa2-final-blending/Cat_full_0.7628.csv')

new13 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7621 (2).csv')
new14 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7623 (2).csv')
new15 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7624 (2).csv')
new16 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7625 (4).csv')
new17 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_0.7627 (1).csv')
new18 = pd.read_csv('../input/kalapa2-final-blending-part2/Cat_full_7629.csv')


a = (cat1['label']+cat2['label']+cat3['label']+cat4['label']+cat5['label']+cat6['label']+cat7['label']+cat8['label'])/8 #*0.6 + 0.4*(sub5['label'] + sub6['label']+sub7['label'])/3
b= (new1['label']+new2['label']+new3['label']+new4['label']+new5['label']+new6['label']+new7['label']+new8['label']+new9['label']+new10['label']+new11['label']+new12['label'] + new13['label']+ new14['label'] + new15['label']+new16['label']+new17['label']+new18['label'])/18

sub = cat1.copy()
sub['label'] = 0.5*a + 0.5*b
sub.to_csv('blending_final.csv', index = False)
sub.head(10)